In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re
from scipy.optimize import linear_sum_assignment

In [ ]:
bodiesDF = pd.read_pickle('synchronization/synchronizeS')
facesDF = pd.read_pickle('facesS')
uniqueNames = np.load('uniqueName.npy')

In [ ]:
noneList = [None] * facesDF.embs.iloc[0].shape[0]
noneArray = [noneList] * bodiesDF.shape[0]
len(bodiesDF['trackX'])
len(noneArray)

In [ ]:
bodiesDF['blur'] = np.nan
bodiesDF['embedding'] =  noneArray

In [ ]:
bodiesDF.head()

In [ ]:
facesDF.head()

In [ ]:
for seq in bodiesDF.rgbSeq.unique():
    bigDist = []
    bodies = bodiesDF[bodiesDF.rgbSeq == seq].reset_index().drop_duplicates(subset='id', keep='first')
    faces = facesDF[facesDF.rgb == seq]
    
    for fi, f in faces.iterrows():
        faceCentroid = (int(f.faceX), int(f.faceY))
        distance = []
        if len(bodies) > 0:
            for bi, b in bodies.iterrows():
                upLeft = tuple((int(b.trackX), int(b.trackY)))
                downRight = tuple((int(b.trackX) + int(b.trackW), int(b.trackY) + int(b.trackH)))
                bodyCentroid = tuple((np.asarray(upLeft) + np.asarray(downRight)) / 2)
                
                inside = (bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2 < (b.trackW)**2
                up = bodyCentroid[1] > faceCentroid[1]
                if inside and up:
                    dist = ((bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2)**(0.5)
                    distance.append(dist)
                else:
                    distance.append(2237)
        bigDist.append(distance)

    if len(bigDist) > 0:
        row_ind, col_ind = linear_sum_assignment(np.asarray(bigDist).T)
        for i in range(len(row_ind)):
            b = bodies.iloc[row_ind[i]]            
            bodiesDF.loc.__setitem__( ((int(b.seq), int(b.id)), 'embedding'), 
                                     np.asarray([faces.iloc[col_ind[i]]['embs']]) ) # Maybe need to be transposed
            bodiesDF.loc.__setitem__(((int(b.seq), int(b.id)), 'blur'), faces.iloc[col_ind[i]]['blurScore'])

In [ ]:
bodiesDF

In [ ]:
bodiesDF.to_pickle('bodyFaceS')

In [ ]:
# For checking
for seq in bodiesDF.rgbSeq.unique():
    #print seq
    bigDist = []
    bodies = bodiesDF[bodiesDF.rgbSeq == seq].reset_index().drop_duplicates(subset='id', keep='first')
    faces = facesDF[facesDF.rgb == seq]
    
    for fi, f in faces.iterrows():
        faceCentroid = (int(f.faceX), int(f.faceY))
        distance = []
        if len(bodies) > 0:
            for bi, b in bodies.iterrows():
                upLeft = tuple((int(b.trackX), int(b.trackY)))
                downRight = tuple((int(b.trackX) + int(b.trackW), int(b.trackY) + int(b.trackH)))
                bodyCentroid = tuple((np.asarray(upLeft) + np.asarray(downRight)) / 2)
                
                inside = (bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2 < (b.trackW)**2                
                up = bodyCentroid[1] > faceCentroid[1]
                if inside and up:
                    dist = ((bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2)**(0.5)
                    distance.append(dist)
                else:
                    distance.append(2237)
        bigDist.append(distance)

    if len(bigDist) > 0:
        row_ind, col_ind = linear_sum_assignment(np.asarray(bigDist).T)
        for i in range(len(row_ind)):
            fileNumber = str("%03d"%seq)
            line = cv2.imread("/home/sabrine/notebook/modified/" + fileNumber + '.jpg')
                            
            for i in range(len(col_ind)):
                b = bodies.iloc[row_ind[i]]
                upLeft = tuple((int(b.trackX), int(b.trackY)))
                downRight = tuple((int(b.trackX) + int(b.trackW), int(b.trackY) + int(b.trackH)))
                bodyCentroid = tuple((np.asarray(upLeft) + np.asarray(downRight)) / 2)

                f = faces.iloc[col_ind[i]]
                faceCentroid = (int(f.faceX), int(f.faceY))
                
                inside = (bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2 < (b.trackW)**2
                line = cv2.circle(line, faceCentroid, (int(b.trackW)), (65,65,65), thickness=3)
                up = bodyCentroid[1] > faceCentroid[1]
                if inside and up:
                    line = cv2.line(line, bodyCentroid, faceCentroid, (255,65,255), thickness=3)

            cv2.imwrite("/home/sabrine/notebook/modified2/" + fileNumber + ".jpg", line)